# Search Tweets
This project is inspired and led by ["How to analyze the sentiment of your own Tweets"](https://blog.twitter.com/developer/en_us/topics/tips/2020/how-to-analyze-the-sentiment-of-your-own-tweets) by Jessica Garson!

In this project, we pulls the Tweets from a certain Twitter account from the past 7 days and gives you a score to let you know exactly how his/her week has been.

In [13]:
import requests
import pandas as pd
import json
import ast
import yaml

# Before you can connect the Twitter API, 
# you’ll need to set up the URL to ensure it has the right fields so you get the right data back. 
# you can get the particular endpoint on https://developer.twitter.com/en/docs/twitter-api/search-overview
# we are using Recent Search Endpoint for the tweets from specific account on the past 7 days 
def create_twitter_url(handle):         # the parameter handle here is the account username
    max_results = 100
    mrf = "max_results={}".format(max_results)
    q = "query=from:{}".format(handle)
    url = "https://api.twitter.com/2/tweets/search/recent?tweet.fields=lang&{}&{}".format(mrf, q)
    return url

# To access the configuration file you created while setting up config.yaml
def process_yaml():
    with open("config.yaml") as file:
        return yaml.safe_load(file)

# To access the bearer token from your config.yaml file
def create_bearer_token(data):
    return data["search_tweets_v2"]["bearer_token"]

# To format the headers to pass in your bearer_token and url
def twitter_auth_and_connect(bearer_token, url):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def no_tweets(res_json):
    if res_json == {"meta": {"result_count": 0}}:
        print("The Twitter handle entered hasn't Tweeted in 7 days.")

def connect_to_azure(data):
    # you can find the endpoint url in your azure project
    azure_url = "https://senti-on-tweet.cognitiveservices.azure.com/" 
    sentiment_url = "{}text/analytics/v2.1/sentiment".format(azure_url)
    # the subscription key we are using here is the API key
    subscription_key = data["azure"]["subscription_key"]
    return sentiment_url, subscription_key

def azure_header(subscription_key):
    return {"Ocp-Apim-Subscription-Key": subscription_key}

def create_document_format(res_json):
    data_only = res_json["data"]
    doc_start = '"documents": {}'.format(data_only)
    str_json = "{" + doc_start + "}"
    dump_doc = json.dumps(str_json)
    doc = json.loads(dump_doc)
    return ast.literal_eval(doc)

def sentiment_scores(headers, sentiment_url, document_format):
    response = requests.post(sentiment_url, headers=headers, json=document_format)
    return response.json()

def mean_score(sentiments):
    sentiment_df = pd.DataFrame(sentiments["documents"])
    return sentiment_df["score"].mean()

def week_logic(week_score):
    if week_score > 0.75 or week_score == 0.75:
        print("This account had a positve week")
    elif week_score > 0.45 or week_score == 0.45:
        print("This account had a neautral week")
    else:
        print("This account had a negative week, I hope it gets better")


# select the paragraph, CTRL + / to (un)comment out the function(s)        
# This is an testing function for providing example results to help you understand the result

# def analysis(handle):
#     url = create_twitter_url(handle)
#     data = process_yaml()
#     bearer_token = create_bearer_token(data)
#     res_json = twitter_auth_and_connect(bearer_token, url)
#     no_tweets(res_json)
#     sentiment_url, subscription_key = connect_to_azure(data)
#     headers = azure_header(subscription_key)
#     document_format = create_document_format(res_json)
#     sentiments = sentiment_scores(headers, sentiment_url, document_format)
#     week_score = mean_score(sentiments)
#     data = pd.DataFrame.from_dict(res_json['data'])
#     print("Base on the ", data.shape[0], "tweets this account post from past 7 days, we get a score of ", week_score)
#     week_logic(week_score)
#     return data



def main():
    handle = input("Enter Account Username: ")
    url = create_twitter_url(handle)
    data = process_yaml()
    bearer_token = create_bearer_token(data)
    res_json = twitter_auth_and_connect(bearer_token, url)
    no_tweets(res_json)
    sentiment_url, subscription_key = connect_to_azure(data)
    headers = azure_header(subscription_key)
    document_format = create_document_format(res_json)
    sentiments = sentiment_scores(headers, sentiment_url, document_format)
    week_score = mean_score(sentiments)
    print("Base on the ", pd.DataFrame.from_dict(res_json['data']).shape[0], "tweets this account post from past 7 days, we get a score of ", week_score)
    week_logic(week_score)

if __name__ == "__main__":
    main()




Enter Account Username: PUMA
Base on the  24 tweets this account post from past 7 days, we get a score of  0.6540741225083669
This account had a neautral week


There are some examples for you to understand what the result you be

In [ ]:
main()

If you want to digging more about the company, you can uncomment the `analysis` function and comment the main function.

In [2]:
# settings for the pandas expression
pd.set_option('display.max_colwidth', None)

### PUMA Sentiment Analysis

In [3]:
puma = analysis("PUMA")
puma

Base on the  24 tweets this account post from past 7 days, we get a score of  0.6540741225083669
This account had a neautral week


,id,lang,text
0,1427292438356545536,en,🖤🤍 @dualipa in the new Suede Mayu 🖤🤍 https://t.co/dQ1ScbqmxY
1,1427276317259706377,en,Greatest. Of. All. Time. 🐐 @usainbolt https://t.co/DVxafLzGeB
2,1427187289709481984,en,RT @DUALIPA: The latest from @PUMA . New Mayu out tomorrow 🖤 shot by Mario Sorrenti #ad https://t.co/70dLcMYUdH
3,1426983973922693120,en,RT @erinasimon: Glad you like our @PUMA RKDO gift package @sjokz!\n\nAppreciate you and thank you for being an awesome esports rep 🔥😃🙌🏽 https…
4,1426950822924111880,und,@LILCOBEY 🤝 #PUMAFam
5,1426950461198938113,und,#SuedeSunday @MELOD1P 🛸💕 https://t.co/9MmkxeWkqx
6,1426948450361483272,en,RT @Oratile011: Top 5 Greatest silhouette ever.
7,1426901382976741377,en,Beautiful from every angle 🧡 #SuedeSunday\n📷: formstripes (IG) https://t.co/YaMCoX7sBL
8,1426892392347676672,en,"RT @HotFreestyle: Nipsey Hussle would’ve been 36 years old today, Happy Birthday &amp; Rest In Peace 🙏🏽🕊🏁 https://t.co/grD03FIVng"
9,1426892327377833985,en,RT @Genius: reminder from nipsey: you're supposed to be here. #verified https://t.co/KOvUz4ZPzI


In [4]:
text_puma = puma[puma['lang']=='en'].drop(['id', 'lang'], axis=1).drop_duplicates('text')
text_puma.count()
text_puma

,text
0,🖤🤍 @dualipa in the new Suede Mayu 🖤🤍 https://t.co/dQ1ScbqmxY
1,Greatest. Of. All. Time. 🐐 @usainbolt https://t.co/DVxafLzGeB
2,RT @DUALIPA: The latest from @PUMA . New Mayu out tomorrow 🖤 shot by Mario Sorrenti #ad https://t.co/70dLcMYUdH
3,RT @erinasimon: Glad you like our @PUMA RKDO gift package @sjokz!\n\nAppreciate you and thank you for being an awesome esports rep 🔥😃🙌🏽 https…
6,RT @Oratile011: Top 5 Greatest silhouette ever.
7,Beautiful from every angle 🧡 #SuedeSunday\n📷: formstripes (IG) https://t.co/YaMCoX7sBL
8,"RT @HotFreestyle: Nipsey Hussle would’ve been 36 years old today, Happy Birthday &amp; Rest In Peace 🙏🏽🕊🏁 https://t.co/grD03FIVng"
9,RT @Genius: reminder from nipsey: you're supposed to be here. #verified https://t.co/KOvUz4ZPzI
10,RT @Kgudie_: Okay my baby just got here 😭😭🤍🤍 https://t.co/VFWZ5qhgp6
11,@iLOVEnewyork83 @andreagrimes Worth it 💚


In [5]:
retweets_puma = text_puma[text_puma['text'].str.contains('RT')]
retweets_puma

,text
2,RT @DUALIPA: The latest from @PUMA . New Mayu out tomorrow 🖤 shot by Mario Sorrenti #ad https://t.co/70dLcMYUdH
3,RT @erinasimon: Glad you like our @PUMA RKDO gift package @sjokz!\n\nAppreciate you and thank you for being an awesome esports rep 🔥😃🙌🏽 https…
6,RT @Oratile011: Top 5 Greatest silhouette ever.
8,"RT @HotFreestyle: Nipsey Hussle would’ve been 36 years old today, Happy Birthday &amp; Rest In Peace 🙏🏽🕊🏁 https://t.co/grD03FIVng"
9,RT @Genius: reminder from nipsey: you're supposed to be here. #verified https://t.co/KOvUz4ZPzI
10,RT @Kgudie_: Okay my baby just got here 😭😭🤍🤍 https://t.co/VFWZ5qhgp6
13,RT @WSeriesRacing: 🔍 a woman's place is 𝐢𝐧 𝐦𝐨𝐭𝐨𝐫𝐬𝐩𝐨𝐫𝐭
19,RT @Bratz: Thursday! 👄👟 @PUMA #bratz https://t.co/vdGDnOExSH
20,RT @Babyyhairz: The Hussle Way \n“Miami Story” August 13\n@PUMA x @themarathonclothing https://t.co/zuWagsy0Yk
22,RT @brkicks: First look at LaMelo Ball’s first signature shoe with Puma called the MB1 😮 @NickDePaula https://t.co/AKzCYxdh6l


### PATAGONIA Sentiment Analysis

In [6]:
# Pull out all the tweets for Patagonia in last 7 days
patagonia = analysis("patagonia")
patagonia

Base on the  65 tweets this account post from past 7 days, we get a score of  0.7800627428751725
This account had a positve week


,id,lang,text
0,1427338949039636490,en,Join Patagonia grantee @UM_Waterkeeper in calling on Montana leaders to protect the state's cold water fisheries for future generations. Click to add your voice.
1,1427334149447340037,en,"For 40 years, a Pennsylvania couple has been trying to escape the health impacts of living near oil and gas activity. Read their story on Patagonia grantee @EHPinfo's blog."
2,1427316269439590425,en,"@dezeerae Hi Des, we're sorry to hear about the disappointment with sizing. We know that sizing has not been inclusive and we're working each season to expand our product sizing to outfit a wider range of body types. You can see what we have available here, https://t.co/Vbs9sIQApT."
3,1427316051033788416,en,"@ckmcdkc Hey Courtney! It's a super rad hoody, right?! That is our new Regenerative Organic Cotton Essential Hoody. It is expected to launch online in early September so check out https://t.co/qXNsy9NEuR around then."
4,1427300813706174465,en,"@ThatVatoJules Hi Julio, we're so sorry to hear you haven't had a great experience with our website. Please DM us so we can further assist with these issues. https://t.co/BQQTdfLPtb"
...,...,...,...
60,1425197325597642752,en,RT @conservationall: Join us TOMORROW at #ORshow for our breakfast event at the Colorado Convention Center (7am MT) or tune in virtually vi…
61,1425195711239491588,en,Patagonia grantee @NatureMuseum is currently recruiting for their Chicago Conservation Corps program. Join them for an orientation to learn about Chicago's key environmental challenges and how you can become a changemaker in your community; click to learn more and RSVP.
62,1425195394200260610,en,"Snake River salmon are on the brink of extinction, putting the ways of life and customs of many Native peoples at stake. Join Native youth in calling for removal of the lower Snake River dams. \n\nSign the petition: https://t.co/vwxgPnvsq6 https://t.co/SSSp9GQinc"
63,1425135415179153411,en,Greg Long taps into momentary bliss in Puerto Escondido. \nVideo: Edwin Morales https://t.co/tZLAtlb4lV


In [7]:
text_patagonia = patagonia.drop(['id', 'lang'], axis=1).drop_duplicates('text')
text_patagonia.count()

text    42
dtype: int64

In [8]:
retweets_patagonia = text_patagonia[text_patagonia['text'].str.contains('RT')]
retweets_patagonia

,text
51,"RT @WHCEQ: Today, we launched the #AmericatheBeautiful Interagency Working Group. Chaired by @BrendaMallory46, @SecDebHaaland, @SecVilsack…"
52,"RT @GrandCanynTrust: Tonight 6PM MDT, join Janene Yazzie, June Lorenzo, and Brett Lee Shelton for a discussion of free, prior, and informed…"
60,RT @conservationall: Join us TOMORROW at #ORshow for our breakfast event at the Colorado Convention Center (7am MT) or tune in virtually vi…


In [9]:
Social_Good = text_patagonia[text_patagonia['text'].str.contains('protect|help|environment')]
print(Social_Good.count())
Social_Good

text    16
dtype: int64


,text
0,Join Patagonia grantee @UM_Waterkeeper in calling on Montana leaders to protect the state's cold water fisheries for future generations. Click to add your voice.
5,"@MariannaFila HI Marianna, we use OnTrac because it helps meet our customers' needs and it is more sustainable to use local shipping options per region. If you currently having issues with one of your orders delivering with OnTrac, DM us so we can help fix the situation! https://t.co/BQQTdfLPtb"
6,Is donating clothes actually helpful? Many shelters in your town will take clothing donations. But they don't want to be burdened with donations that aren't clean or useful. Comics journalist Sarah Mirk learns how to donate with dignity: https://t.co/kj4VoNEqwR https://t.co/WaOMSj0kX1
10,"@MackFordFan98 Perfect, we are incredibly happy to hear that Mack! If you have any questions or concerns during the exchange process, please let us know and we will be more than happy to help. 😊"
27,"After 40 years of supporting families by offering onsite childcare, we’ve learned we can create a more equitable and productive culture, help retain and attract top talent and recoup most of the cost. Thanks for creating and leading on a national family first policy agenda @VP."
42,Take action with Patagonia grantee @WFA_WildFarm as they work to ensure that organic standard rules include the protection of native ecosystems. Click to add your name.
43,Join Patagonia grantee @WFA_WildFarm in calling on the National Organic Program to ensure that organic standard rules include the protection of native ecosystems. Click to add your name.
48,Join Patagonia grantee @NRCM in calling on Maine leaders to support permanent protection for the Arctic National Wildlife Refuge. Click to take action.
49,Join Patagonia grantee @NRCM in calling on Maine leaders to help permanently protect the Arctic Refuge from oil and gas drilling.
50,Join Patagonia grantee @NRCM in calling on Maine leaders to help permanently protect the Arctic National Wildlife Refuge from oil and gas drilling.


In [10]:
reply = text_patagonia[text_patagonia['text'].str.contains('Hi|Thank you|Hey')]
print(reply.count())
reply

text    7
dtype: int64


,text
2,"@dezeerae Hi Des, we're sorry to hear about the disappointment with sizing. We know that sizing has not been inclusive and we're working each season to expand our product sizing to outfit a wider range of body types. You can see what we have available here, https://t.co/Vbs9sIQApT."
3,"@ckmcdkc Hey Courtney! It's a super rad hoody, right?! That is our new Regenerative Organic Cotton Essential Hoody. It is expected to launch online in early September so check out https://t.co/qXNsy9NEuR around then."
4,"@ThatVatoJules Hi Julio, we're so sorry to hear you haven't had a great experience with our website. Please DM us so we can further assist with these issues. https://t.co/BQQTdfLPtb"
8,"@djbuffnstuff Hi Craig, we're so sorry to hear about trouble with your order arriving. Please get back to us privately so we can further assist you. https://t.co/BQQTdfLPtb"
9,"@JKBAD Hey, thanks for bringing this to our attention. Unfortunately, it is a scam. We'll DM you with next steps."
26,@Joshua7Berry Hey Joshua! We currently have inventory of the M's Lightweight Synchilla Snap-T Fleece Pullover in Oatmeal Heather online https://t.co/Ts4OBvGIo5. Feel free to DM us with specific questions! https://t.co/BQQTdfLPtb
59,"@me_idealist Hey there, this is the address: 189 Stag Hill Rd, Mahwah, NJ 07430-1041"


### JanSport Sentiment Analysis

In [11]:
# Pull out all the tweets for JanSport in last 7 days
JanSport= analysis("JanSport")
JanSport

The Twitter handle entered hasn't Tweeted in 7 days.


KeyError: 'data'

### Columbia Sentiment Analysis

In [12]:
columbia = analysis("Columbia1938")
columbia

Base on the  2 tweets this account post from past 7 days, we get a score of  0.9218319654464722
This account had a positve week


,id,lang,text
0,1426283619644026891,en,"This 106-mile course doles out 32,940 feet of elevation gain in a series of brutal climbs, although each one is set against a stunning panoramic backdrop. ​\n\nFind out what makes the @UTMBMontBlanc so badass:​\nhttps://t.co/Y3eorH2wFU"
1,1425215856708132870,en,"To help you make the right choice, we’ve listed seven of the most important qualities to keep in mind when you’re shopping for a backpack. \nhttps://t.co/P1ByHRq3Fu"
